# Random Forest running with only VVR_1 and VVR_2

Import libraries. 

In [20]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

# Import data 

In [30]:
# Load the data into pandas dataframes
X_train_12 = pd.read_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_12.csv')
y_train_12 = pd.read_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_train_12.csv')
X_test_12 = pd.read_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test_12.csv')
y_test = pd.read_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_test.csv')

X_train_12 = X_train_12.drop(columns='Unnamed: 0', axis=1)
y_train_12 = y_train_12.drop(columns='Unnamed: 0', axis=1)
X_test_12 = X_test_12.drop(columns='Unnamed: 0', axis=1)
y_test = y_test.drop(columns='Unnamed: 0', axis=1)

print(X_train_12.columns)
print(X_test_12.columns)


Index(['Sum_1', 'Sum_2'], dtype='object')
Index(['Sum_1', 'Sum_2'], dtype='object')


In [31]:
# Display the first few rows of each dataframe to verify the loading
print("X_train_12:")
display(X_train_12.head())

print("\ny_train_12:")
display(y_train_12.head())

print("\nX_test_12:")
display(X_test_12.head())

print("\ny_test:")
# print(y_test)
display(y_test.head())

# Convert the DataFrame to a 1-dimensional NumPy array
y_train_12 = y_train_12.values.ravel()
y_test = y_test.values.ravel()

X_train_12:


,Sum_1,Sum_2
0,12.0,13.0
1,17.0,16.0
2,16.0,12.0
3,8.0,8.0
4,9.0,8.0



y_train_12:


,VVR_group
0,1
1,1
2,1
3,0
4,0



X_test_12:


,Sum_1,Sum_2
0,9.0,11.0
1,11.0,10.0
2,11.0,10.0
3,8.0,8.0
4,8.0,8.0



y_test:


,VVR_group
0,0
1,0
2,0
3,0
4,0


# Model with inner and outer split
https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html  

New.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)

# Create the pipeline with RFE and the model
# rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=2)
rfe = RFE(estimator=DecisionTreeClassifier())
model = RandomForestClassifier()
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])

param_grid = {
    'm__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'm__class_weight': [{0: 0.7549019607843137, 1: 1.4807692307692308}, None],  # Class weights
    'm__max_depth': [None, 10, 20],  # Maximum depth of the tree
    # 's__n_features_to_select': [5, 20]  # Number of features to select with RFE

}

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=2
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_train_12, y_train_12, cv=outer_cv, n_jobs=2)
print(
    "The mean score using nested cross-validation is: "
    f"{test_score.mean():.3f} ± {test_score.std():.3f}"
)

# Print shape
print(X_train_12.shape)
print(y_train_12.shape)

# Fit model to training data to get best parameters
model.fit(X_train_12, y_train_12)

# Print best parameters
print(model.best_params_)

# Print the features that were selected with RFE

# Evaluate on the test set
best_model = model.best_estimator_
y_pred = best_model.predict(X_test_12)

# Print shape
print(y_test.shape)
print(y_pred.shape)

# Print classification report 
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

# Calculate and print the AUC-PR
y_proba = best_model.predict_proba(X_test_12)[:, 1]  # Get the probabilities for the positive class
precision, recall, _ = precision_recall_curve(y_test, y_proba)
auc_pr = auc(recall, precision)
print(f"AUC-PR: {auc_pr:.3f}")

# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
# print(metrics.auc(fpr, tpr))


The mean score using nested cross-validation is: 0.699 ± 0.104
(77, 2)
(77,)
{'m__class_weight': None, 'm__max_depth': None, 'm__n_estimators': 100, 's__n_features_to_select': 5}
(34,)
(34,)

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84        23
           1       0.71      0.45      0.56        11

    accuracy                           0.76        34
   macro avg       0.75      0.68      0.70        34
weighted avg       0.76      0.76      0.75        34

[[21  2]
 [ 6  5]]
AUC-PR: 0.601


# Optimized code 

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_validate, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_curve, auc

def build_pipeline():
    """Builds the machine learning pipeline with RFE and RandomForestClassifier."""
    rfe = RFE(estimator=DecisionTreeClassifier(random_state=0))
    model = RandomForestClassifier(random_state=0)
    return Pipeline(steps=[('feature_selection', rfe), ('classifier', model)])

def get_param_grid():
    """Returns the hyperparameter grid for GridSearchCV."""
    return {
        'classifier__n_estimators': [100, 200, 300],  # Number of trees in the forest
        'classifier__class_weight': [{0: 0.75, 1: 1.48}, None],  # Class weights
        'classifier__max_depth': [None, 10, 20],  # Maximum depth of the tree
        'feature_selection__n_features_to_select': [5, 10, 20, 40, 80]  # Number of features to select with RFE
    }

def perform_nested_cv(X, y, random_state=0):
    """Performs nested cross-validation and returns the test scores."""
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
    outer_cv = KFold(n_splits=3, shuffle=True, random_state=random_state)
    
    pipeline = build_pipeline()
    param_grid = get_param_grid()
    
    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=2)
    outer_cv_results = []

    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        grid_search.fit(X_train, y_train)
        
        # Get the scores from the inner cross-validation
        inner_cv_results = grid_search.cv_results_['mean_test_score']
        inner_cv_std = grid_search.cv_results_['std_test_score']
        
        outer_cv_results.append((inner_cv_results.mean(), inner_cv_std.mean(), grid_search.best_estimator_))
    
    outer_cv_means = [result[0] for result in outer_cv_results]
    outer_cv_stds = [result[1] for result in outer_cv_results]
    
    print(f"Mean scores for each outer fold: {outer_cv_means}")
    print(f"Standard deviation for each outer fold: {outer_cv_stds}")
    print(f"The overall mean score using nested cross-validation is: {np.mean(outer_cv_means):.3f} ± {np.std(outer_cv_means):.3f}")
    
    return outer_cv_results

def plot_curves(y_test, y_proba, y_pred):
    """Plots PR and ROC curves."""
    # Plot Precision-Recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    auc_pr = auc(recall, precision)
    plt.figure()
    plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {auc_pr:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='best')
    plt.show()

    # Plot ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_roc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_roc:.3f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt.show()

def evaluate_model(best_model, X_test, y_test):
    """Evaluates the best model on the test set and plots the curves."""
    print("\nEvaluating best model on test set")
    y_test_pred = best_model.predict(X_test)
    
    print("\nClassification Report on Test Set:")
    print(classification_report(y_test, y_test_pred))
    print(confusion_matrix(y_test, y_test_pred))
    
    y_test_proba = best_model.predict_proba(X_test)[:, 1]
    plot_curves(y_test, y_test_proba, y_test_pred)
    
    precision, recall, _ = precision_recall_curve(y_test, y_test_proba)
    auc_pr = auc(recall, precision)
    print(f"Test AUC-PR: {auc_pr:.3f}")

# Example usage with your data
random_seed = 42

# Print the shape of your training and test data
print("Shape of X_train_12:", X_train_12.shape)
print("Shape of y_train_12:", y_train_12.shape)

# Convert y_train_12 and y_test to pandas Series after raveling them
y_train_12 = pd.Series(y_train_12.values.ravel())

# Convert y_test to pandas Series after raveling it
y_test = pd.Series(y_test.values.ravel())

print("Shape of X_test_12:", X_test_12.shape)
print("Shape of y_test:", y_test.shape)

cv_results = perform_nested_cv(X_train_12, y_train_12, random_state=random_seed)

# Use the best model from the cross-validation for evaluation on the test set
best_model = max(cv_results, key=lambda x: x[0])[2]

evaluate_model(best_model, X_test_12, y_test)

Shape of X_train_12: (77, 2)
Shape of y_train_12: (77,)


AttributeError: 'numpy.ndarray' object has no attribute 'values'

# Confusion matrix 

In [19]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

font = 'Times New Roman'
size = 18

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Set the style to a modern style
plt.style.use('seaborn-darkgrid')

# Create a custom diverging color map
colors = sns.color_palette("coolwarm", as_cmap=True)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=colors, cbar=False,
            annot_kws={'fontname': font, 'fontsize': size})  # Set font and size for numbers
plt.title('Random Forest', fontname=font, fontsize=size)
plt.xlabel('Predicted', fontname=font, fontsize=size)
plt.ylabel('Actual', fontname=font, fontsize=size)

plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [30, 34]